In [1]:
!pip install --quiet optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.6 MB/s eta 0:00:00


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import optuna

In [27]:
# Define the CNN model architecture
class Net(nn.Module):
    def __init__(self, n_conv_layers, n_filters, n_fc_layers, n_neurons, dropout_rate):
        super(Net, self).__init__()
        layers = []
        in_channels = 3
        image_size = 32

        # Add convolutional layers
        for i in range(n_conv_layers):
            layers.append(nn.Conv2d(in_channels, n_filters, kernel_size=3, padding=1))
            layers.append(nn.ReLU())
            layers.append(nn.MaxPool2d(2, 2))
            in_channels = n_filters

        # Calculate the final image size after convolution and pooling
        image_size = image_size // (2 ** n_conv_layers)

        # Add fully connected layers
        layers.append(nn.Flatten())
        in_features = n_filters * image_size * image_size  # Update in_features for the first linear layer
        for i in range(n_fc_layers):
            layers.append(nn.Linear(in_features, n_neurons))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(p=dropout_rate))
            in_features = n_neurons  # Update in_features for the next layer

        layers.append(nn.Linear(in_features, 100))  # 100 output classes for CIFAR-100
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Function to get the CIFAR-100 data loaders
def get_data_loaders(batch_size):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

    testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

    return trainloader, testloader

# Function to train and evaluate the model
def train_and_evaluate(model, optimizer, criterion, trainloader, testloader, device, trial, epochs=10):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        train_total = 0
        train_correct = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            running_loss += loss.item()

        ## Calculate accuracy for training phase
        train_accuracy = train_correct / train_total
        ## NORMAL running_loss over batches
        running_loss /= len(trainloader)

        model.eval()
        test_total = 0
        test_correct = 0
        with torch.no_grad():
            for data in testloader:
                images, labels = data[0].to(device), data[1].to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()

        ## Calculate accuracy for test phase
        test_accuracy = test_correct / test_total

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.3f}, TrAccuracy: {100 * train_accuracy:.2f}%, TeAccuracy: {100 * test_accuracy:.2f}")

        # Report intermediate result to Optuna for pruning
        trial.report(test_accuracy, epoch)

        # Handle pruning based on the intermediate result
        if test_accuracy < 0.06:
            print("Trial pruned due to low accuracy.")
            raise optuna.TrialPruned()

    return 1 - test_accuracy


# Define the optimization objective
def objective(trial):
    # Set up the hyperparameters to optimize
    n_conv_layers = trial.suggest_int('n_conv_layers', 1, 4)
    n_filters = trial.suggest_int('n_filters', 16, 64)
    n_fc_layers = trial.suggest_int('n_fc_layers', 1, 3)
    n_neurons = trial.suggest_int('n_neurons', 16, 64)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-3, 1e-2)
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD'])

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Get data loaders
    trainloader, testloader = get_data_loaders(batch_size=128)

    # Create the model
    model = Net(n_conv_layers, n_filters, n_fc_layers, n_neurons, dropout_rate)

    # Set up the optimizer and loss criterion
    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
    else:
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    criterion = nn.CrossEntropyLoss()

    # Train and evaluate the model
    error_rate = train_and_evaluate(model, optimizer, criterion, trainloader, testloader, device, trial)

    return error_rate


In [28]:
# Set up Optuna study
study = optuna.create_study(direction='minimize')

try:
    study.optimize(objective, timeout=3600, n_jobs=1)
except optuna.exceptions.TrialPruned as e:
    print("Trial was pruned.")


[I 2023-07-01 12:00:13,476] A new study created in memory with name: no-name-ead9a5fb-e564-442e-bd3f-e6c375b47347
<ipython-input-27-ae6992357a73>:108: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 0.5)
<ipython-input-27-ae6992357a73>:109: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-3, 1e-2)


Files already downloaded and verified
Files already downloaded and verified


[I 2023-07-01 12:02:01,983] Trial 0 pruned. 


Epoch 1/10, Loss: 4.609, TrAccuracy: 0.89%, TeAccuracy: 0.69
Trial pruned due to low accuracy.
Files already downloaded and verified
Files already downloaded and verified


[I 2023-07-01 12:02:58,526] Trial 1 pruned. 


...

Epoch 1/10, Loss: 4.575, TrAccuracy: 1.78%, TeAccuracy: 3.43
Trial pruned due to low accuracy.
Files already downloaded and verified
Files already downloaded and verified


[I 2023-07-01 12:55:16,645] Trial 11 pruned. 


Epoch 1/10, Loss: 4.395, TrAccuracy: 2.76%, TeAccuracy: 5.41
Trial pruned due to low accuracy.
Files already downloaded and verified
Files already downloaded and verified
Epoch 1/10, Loss: 4.127, TrAccuracy: 6.50%, TeAccuracy: 13.07
Epoch 2/10, Loss: 3.619, TrAccuracy: 13.98%, TeAccuracy: 18.94
Epoch 3/10, Loss: 3.378, TrAccuracy: 18.26%, TeAccuracy: 23.52
Epoch 4/10, Loss: 3.208, TrAccuracy: 21.31%, TeAccuracy: 24.90
Epoch 5/10, Loss: 3.074, TrAccuracy: 23.66%, TeAccuracy: 25.86
Epoch 6/10, Loss: 2.963, TrAccuracy: 25.60%, TeAccuracy: 30.01
Epoch 7/10, Loss: 2.870, TrAccuracy: 27.47%, TeAccuracy: 30.98
Epoch 8/10, Loss: 2.794, TrAccuracy: 29.17%, TeAccuracy: 33.15
Epoch 9/10, Loss: 2.726, TrAccuracy: 30.34%, TeAccuracy: 32.47


[I 2023-07-01 13:18:08,410] Trial 12 finished with value: 0.6644 and parameters: {'n_conv_layers': 3, 'n_filters': 49, 'n_fc_layers': 1, 'n_neurons': 49, 'dropout_rate': 0.17971506180255348, 'learning_rate': 0.0017326535267501914, 'optimizer': 'RMSprop'}. Best is trial 12 with value: 0.6644.


Epoch 10/10, Loss: 2.678, TrAccuracy: 31.02%, TeAccuracy: 33.56


In [29]:
# Get the best hyperparameters and error rate
best_params = study.best_params
best_error = study.best_value

print("Best Hyperparameters:", best_params)
print("Best Error Rate:", best_error)

Best Hyperparameters: {'n_conv_layers': 3, 'n_filters': 49, 'n_fc_layers': 1, 'n_neurons': 49, 'dropout_rate': 0.17971506180255348, 'learning_rate': 0.0017326535267501914, 'optimizer': 'RMSprop'}
Best Error Rate: 0.6644
